In [1]:
%pylab
from sys import argv
import h5py
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os.path as osp

# m83_data = np.loadtxt("m83_clusters.dat")
# m83_masses = 10**m83_data[:,8]
# m83_masses_max = 10**m83_data[:,11]
# m83_masses_min = 10**m83_data[:,14]
# m83_RGC = 10**m83_data[:,5]
# m83_class = m83_data[:,6]
# m83_age = 10**m83_data[:,7]
# cut = (m83_masses_max-m83_masses_min < m83_masses) * (m83_class == 1) * (m83_age > 1e7) * (m83_RGC >1e3)
# m83_masses = m83_masses[cut]

m51_data = np.loadtxt("m51_legus.tab")
m51_mass = m51_data[:,20]
m51_class = m51_data[:,-2]
m51_mass = m51_mass[m51_class == 1]
#plt.loglog(np.sort(m51_mass), np.arange(len(m51_mass))[::-1]+1)
#plt.show()

m31_data = np.loadtxt("m31.dat")
m31_age = 10**m31_data[:,0]
m31_m = 10**m31_data[:,1]
m31_m = m31_m[(m31_age > 1e7)*(m31_age < 2e8)]

/home/mike/.conda/envs/idp/lib/python3.6/site-packages/IPython/core/pylabtools.py:269: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib
/home/mike/.conda/envs/idp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
data = np.loadtxt("ClusterPopulation_1_reduced.dat")
a = np.loadtxt("snapshot_times.txt")[1:,1]
t = np.loadtxt("snapshot_times.txt")[1:,3]
n = np.loadtxt("snapshot_times.txt")[1:,0]
dt = np.loadtxt("snapshot_times.txt")[1:,-1]
sfrdata = np.loadtxt("m12i_res7100_mhdcv_sfr.dat")[1:-10]
sfrdict=  dict(zip(sfrdata[:,0], sfrdata[:,1]))
mdict = dict(zip(sfrdata[:,0], sfrdata[:,3]))
F = h5py.File("ClusterPopulation_1.hdf5",'r')
#redshifts = 1/a - 1
redshifts = np.loadtxt("snapshot_times.txt")[:,2]
times = np.loadtxt("snapshot_times.txt")[:,3]

In [3]:



fig, axes = plt.subplots(1,3, figsize=(8,4))
for f in "ClusterPopulation_1.hdf5",:
    F = h5py.File(f,'r')
    filt = np.array(F["Mass"]) > 1e4
    Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
    M = np.array(F["Mass"])[filt]
    tf = 13.79874688 - np.array(F["FormationTime"])[filt]
    axes[0].scatter(tf, Z,s=.1,color='black', marker='o',alpha=0.3)
    axes[0].set_xlabel("Age (Gyr)")
    axes[0].set_ylabel("Metallicity")
    axes[0].set_ylim(-4,1)
    axes[2].scatter(tf, M,s=.1,color='black', marker='o',alpha=0.3)
    axes[2].set_xlabel("Age (Gyr)")
    axes[2].set_ylabel("Mass ($M_\odot$)")
    axes[2].set_ylim(1e4,1e7)
    axes[2].set_yscale('log')
    axes[1].scatter(M, Z, s=.1, color='black', marker='o',alpha=0.3)
    axes[1].set_xlabel("Mass ($M_\odot$)")
    axes[1].set_xscale('log')
    axes[1].set_ylabel("Metallicity")
    axes[1].set_ylim(-4,1)
#    plt.xscale('log')
    
plt.tight_layout()
plt.savefig("Mass_Age_Metallicity.png", bbox_inches='tight', dpi=400)


/home/mike/.conda/envs/idp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [156]:
from palettable.colorbrewer.qualitative import Set2_4
agebins = t.max() * np.linspace(0.0,1,200)

F = h5py.File("ClusterPopulation_1.hdf5",'r')
tf = np.array(F["FormationTime"])
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax2 = ax.twiny()
ax.set_prop_cycle('color', Set2_4.mpl_colors)

for masscut in 0, 1e4, 1e5, 1e6:
    print(masscut)
    filt = np.array(F["Mass"]) > masscut
    #agebins = np.concatenate([[0],t])
    Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
    M = np.array(F["Mass"])[filt]
    tf = np.array(F["FormationTime"])[filt]
    
    mdot_cluster = np.histogram(tf, agebins, weights=M)[0] / np.diff(1e9*agebins)

    sfr_snap = sfrdata[:,3] / (dt*1e6)
    mdot_star = np.histogram(t, agebins, weights=sfrdata[:,3])[0] / np.diff(agebins) / 1e9
    CFE = mdot_cluster / mdot_star
    
    if masscut == 1e6:
        ax.fill_between(agebins[1:], CFE, np.zeros_like(CFE), alpha=1, label=("All bound clusters" if masscut==0 else r"$M_\mathrm{cl}>10^{%d}M_\odot$"%int(np.log10(masscut)+0.5)), color='red')
        ax.plot(agebins[1:], CFE,color='red',lw=0.3)
    else:
        ax.fill_between(agebins[1:], CFE, np.zeros_like(CFE), alpha=1, label=("All bound clusters" if masscut==0 else r"$M_\mathrm{cl}>10^{%d}M_\odot$"%int(np.log10(masscut)+0.5)))
        ax.plot(agebins[1:], CFE,color='black',lw=0.3)
ax.set_ylim(1e-4,3)
ax.legend(labelspacing=0)
ax.set_yscale('log')
ax.set_xlabel("Cosmic Time [Gyr]")
ax.set_ylabel("Cluster Formation Efficiency $\Gamma$")
ax.set_xlim(0,13.8)
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(np.sort(np.interp(np.arange(0,6), np.sort(redshifts), times[redshifts.argsort()])))
ax2.set_xticklabels([str(a) for a in np.arange(0,6)[::-1]])
ax2.set_xlabel(r"$z$")
ax2.minorticks_off()
ax2.set_xlim(0,13.8)
plt.savefig("Time_vs_Gamma.pdf", bbox_inches='tight')

0


/home/mike/.conda/envs/idp/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


10000.0
100000.0
1000000.0


In [155]:
np.interp(5, np.sort(redshifts), times[redshifts.argsort()])

1.2089659

In [2]:
from palettable.colorbrewer.qualitative import Set2_4
from scipy.stats import binned_statistic

agebins = t.max() * np.linspace(0.0,1,600)

F = h5py.File("ClusterPopulation_1.hdf5",'r')
tf = np.array(F["FormationTime"])
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax2 = ax.twiny()
#ax.set_prop_cycle('color', Set2_4.mpl_colors)

#for masscut in 0, 1e4, 1e5, 1e6:
#    print(masscut)
filt = np.array(F["Mass"]) > 0
    #agebins = np.concatenate([[0],t])
Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
M = np.array(F["Mass"])[filt]
tf = np.array(F["FormationTime"])[filt]

massmed = lambda x: np.interp(x.sum()/2, np.sort(x).cumsum(), np.sort(x))

mmax_cluster = binned_statistic(tf, M, statistic='max', bins=agebins)[0]
#mmed_cluster = binned_statistic(tf, M, statistic=massmed, bins=agebins)[0]
  
ax.plot(agebins[1:], mmax_cluster, color='black')
#ax.plot(agebins[1:], mmed_cluster)
#ax.set_ylim(1e-4,3)
ax.set_xlim(0,13.8)

ax.legend(labelspacing=0)
ax.set_yscale('log')
ax.set_xlabel("Cosmic Time [Gyr]")
ax.set_ylabel(r"Maximum Cluster Mass [$M_\odot$]")
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(np.sort(np.interp(np.arange(0,6), np.sort(redshifts), times[redshifts.argsort()])))
ax2.set_xticklabels([str(a) for a in np.arange(0,6)[::-1]])
ax2.set_xlabel(r"$z$")
ax2.set_xlim(0,13.8)
ax2.minorticks_off()
plt.savefig("Time_vs_Mcluster.pdf", bbox_inches='tight')

NameError: name 't' is not defined

In [5]:
from palettable.colorbrewer.qualitative import Set2_4
from scipy.stats import binned_statistic

agebins = t.max() * np.linspace(0.0,1,int(t.max()/0.1+1))

F = h5py.File("ClusterPopulation_1.hdf5",'r')
tf = np.array(F["FormationTime"])
dt_tf = np.interp(tf, t, dt)

plt.clf()
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)

tf = np.array(F["FormationTime"])
M = np.array(F["Mass"])
for i in range(len(agebins)-1):    
    filt = (tf > agebins[i])*(tf<=agebins[i+1])*(M>1e3)
    mbin = M[filt]
    ax.plot(np.sort(mbin),(np.arange(len(mbin))[::-1]+1), color='black',alpha=0.05, label=("$100\mathrm{Myr}$ age bins" if i==0 else None))
    #agebins = np.concatenate([[0],t])
#Z = np.log10(F["Metallicity"])[filt] - np.log10(0.02)
ax.plot(np.sort(M[M>1e3]), (np.arange(len(M[M>1e3]))[::-1]+1),label="Total")

# plot m83 CDF from Adamo 2015
m83_masses = 10**np.loadtxt("m83_mass_cdf_adamo2015.csv",delimiter=',')[:,0]
m83_cdf = np.loadtxt("m83_mass_cdf_adamo2015.csv",delimiter=',')[:,1]
m83_cdf /= m83_cdf.min()
ax.plot(np.sort(m83_masses), m83_cdf[m83_masses.argsort()],color='red', label="M83 (Adamo+15)")

#  M51 CDF from Messa 2018
ax.plot(np.sort(m51_mass), np.arange(len(m51_mass))[::-1]+1, label="M51 (Messa+18)")

# M31 CDF from Johnson 2016
ax.plot(np.sort(m31_m), np.arange(len(m31_m))[::-1]+1, label="M31 (Johnson+18)")

ax.text(1.1e6,2000,"$\propto M_\mathrm{cl}^{-2}$", rotation=-60)
ax.text(6e5,350,"$\propto M_\mathrm{cl}^{-1}$", rotation=-37)
ax.set_xlim(1e3,1e7)
ax.set_ylim(1,1e5)
ax.plot([5e5,5e6],[3000,300],color='black',ls='dotted')
ax.plot([5e5,5e6],[30000,300],color='black',ls='dotted')
ax.legend(labelspacing=0,frameon=True,loc=2)
ax.set_ylabel(r"$N\left(>M_\mathrm{cl}\right)$")
ax.set_xlabel(r"$M_\mathrm{cl}$ [$M_\odot$]")
ax.set_yscale('log')
ax.set_xscale('log')
#ax.set_xlabel("Cosmic Time [Gyr]")
#ax.set_ylabel(r"Maximum Cluster Mass [$M_\odot$]")
plt.savefig("Time_vs_MassFunction.pdf", bbox_inches='tight')

In [35]:
plt.loglog(np.sort(m83_masses), m83_cdf[m83_masses.argsort()])
plt.show()

/home/mike/.conda/envs/idp/lib/python3.6/tkinter/__init__.py:745: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [ ]:
Myoung = M[tf>13.7]
plt.plot(M[M.argsort()], M[M.argsort()].cumsum())
plt.xscale('log')

In [ ]:
corr =np.average(1/dt[1:]) * dt

In [ ]:
M.min()

In [ ]:
len(M)/1e6

In [ ]:
t